# Загрузка и обработка сырых данных

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10, 8)
sns.set_palette('flare')

In [2]:
all_orders = pd.read_excel(
    "../data/BurgerKing_TimeSeries_TEST.xlsx", sheet_name="TimeSeries"
)

In [3]:
all_orders

,key,ds,y
0,1,2019-06-10,1
1,1,2019-06-11,14
2,1,2019-06-12,332
3,1,2019-06-13,770
4,1,2019-06-14,840
...,...,...,...
10011,5,2023-07-27,982
10012,5,2023-07-28,949
10013,5,2023-07-29,941
10014,5,2023-07-30,996


In [4]:
all_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10016 entries, 0 to 10015
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   key     10016 non-null  int64         
 1   ds      10016 non-null  datetime64[ns]
 2   y       10016 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 234.9 KB


In [5]:
all_orders.describe()

,key,ds,y
count,10016.000000,10016,10016.000000
mean,3.232528,2020-09-02 06:04:53.290734592,674.183506
min,1.000000,2016-01-02 00:00:00,1.000000
25%,2.000000,2019-05-20 00:00:00,470.000000
50%,3.000000,2020-10-27 00:00:00,639.000000
75%,4.000000,2022-03-13 06:00:00,823.000000
max,5.000000,2023-07-31 00:00:00,2268.000000
std,1.351298,NaN,324.145112


In [6]:
all_orders["key"].unique()

array([1, 2, 3, 4, 5], dtype=int64)

#### Выводы по базовому анализу:
- В срырых данных отсутствуют явные (NaN) пропуски.
- `key` - метка конкретного ресторана, всего 5 ресторанов.
- `ds` - дни в которые производились заказы (рестораны работали). В совокупности по ресторанам доступны данные с 2 января 2016 года по 31 июля 2023 года.
- `y` - количество заказов в конкретный день для конкретного ресторана. Минимальное количество заказов 1, максимальное 2268.

### Трансформация данных
- Необходимо использовать даты в качестве индексов
- Добавить пропущенные даты (дни в которые ресторан не работал)
- Сохранение файлов отдельно по каждому ресторану

In [7]:
def get_restaurant_by_num(df, num):
    rest_df = df[df["key"] == num].copy()
    rest_df.drop(columns="key", inplace=True)
    rest_df.columns = ["Date", "Orders"]
    return rest_df

In [8]:
def transform_to_ts(df):
    new_df = df.copy()
    new_df.set_index("Date", inplace=True)
    full_date_range = pd.date_range(
        start=new_df.index.min(), end=new_df.index.max(), freq="D"
    )
    new_df = new_df.reindex(full_date_range)
    return new_df

In [9]:
rest_list = list(all_orders["key"].unique())

for rest_num in rest_list:
    rest_df = get_restaurant_by_num(all_orders, rest_num)
    rest_ts = transform_to_ts(rest_df)

    rest_ts.to_csv(f"../data/restaurant_{rest_num:02}_ts.csv")

### Обработка календаря

In [10]:
calendar = pd.read_excel(
    "../data/BurgerKing_TimeSeries_TEST.xlsx", sheet_name="Calendar"
)
calendar

,day_id,week_id,month_id,year_id,day_year,day_week,day_number,week_number,month_number,quarter_number,is_holiday,is_warm,is_black_friday
0,2016-01-01,201553,201601,2016,1,5,1,53,1,1,1,0,0
1,2016-01-02,201553,201601,2016,2,6,2,53,1,1,0,0,0
2,2016-01-03,201553,201601,2016,3,7,3,53,1,1,0,0,0
3,2016-01-04,201601,201601,2016,4,1,4,1,1,1,1,0,0
4,2016-01-05,201601,201601,2016,5,2,5,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3283,2024-12-27,202452,202412,2024,362,5,27,52,12,4,0,0,0
3284,2024-12-28,202452,202412,2024,363,6,28,52,12,4,0,0,0
3285,2024-12-29,202452,202412,2024,364,7,29,52,12,4,0,0,0
3286,2024-12-30,202501,202412,2024,365,1,30,1,12,4,1,0,0


In [11]:
calendar.set_index("day_id", inplace=True)
calendar.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3288 entries, 2016-01-01 to 2024-12-31
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   week_id          3288 non-null   int64
 1   month_id         3288 non-null   int64
 2   year_id          3288 non-null   int64
 3   day_year         3288 non-null   int64
 4   day_week         3288 non-null   int64
 5   day_number       3288 non-null   int64
 6   week_number      3288 non-null   int64
 7   month_number     3288 non-null   int64
 8   quarter_number   3288 non-null   int64
 9   is_holiday       3288 non-null   int64
 10  is_warm          3288 non-null   int64
 11  is_black_friday  3288 non-null   int64
dtypes: int64(12)
memory usage: 333.9 KB


In [12]:
calendar.to_csv(f"../data/calendar_ts.csv")